하추치 뉴스를 분석할 수 있지만, 때로는 미리 분서하려는 모든 날의 뉴스로부터 단어사전을 만들어둘 필요가 있음. 
- 10일치의 뉴스에 대해서 명사를 모두 추출한 뒤, universal dictionary를 만드는 것
- dictionary를 이용하여 2016-10-20 뉴스에 대해 뉴스의 군집화를 수행한 뒤, 각 군집의 키워드를 추출함으로 그날 뉴스의 핫키워드를 추출

각 뉴스 문서 별로 term frquency matrix를 만들고, 이를 이용하여 곧바로 Regularized Logistic Regression을 통해 키워드를 선택

In [1]:
EXTRACT_NOUN_SET = True
noun_dictionary_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/noun_dictionary.txt'

TOKENIZE_2016_1020 = True
normed_corpus_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/news/2016-10-20_article_all_normed.txt'
tokenized_corpus_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/news/2016-10-20_article_all_normed_nountokenized.txt'

CREATE_TERM_FREQUENCY_MATRIX = True
x_2016_1020_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-20_noun_x.mm'
vectorizer_2016_1020_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-20_noun_vectorizer.pkl'

corpus_10days의 10일치 뉴스에 대해, 각 날의 기사마다 명사를 추출하여, 이를 nouns_dictionary에 누적하여 저장해보자

아래의 구문을 통해 하루의 뉴스기사를 분석할 때마다, 명사사전의 크기가 얼마나 커져가는지도 확인할 수 있음

피클링을 하지 않고 명사 사전을 텍스트 파일로 저장. 피클링은 데이터를 binary로 저장하기 때문에 파일로 직접 읽을 수가 없음. 하지만 텍스트 파일로 저장하면 눈으로 확인할 수 있고, 다른 프로그래밍언어로 읽을 수도 있음

In [2]:
if EXTRACT_NOUN_SET:
    import glob
    from soynlp.noun import LRNounExtractor
    from soynlp.utils import DoublespaceLineCorpus as Corpus
    
    corpus_fnames = glob.glob('/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/news/*_article_all_normed.txt')
    print('num corpus = %d'%len(corpus_fnames))
    
    nouns_dictionary = set()
    
    for corpus_fname in corpus_fnames:
        news_corpus = Corpus(corpus_fname, iter_sent=True)
        noun_extractor = LRNounExtractor()
        nouns = noun_extractor.train_extract(news_corpus)
        
        nouns_dictionary.update(set(nouns.keys()))
        corpus_name = corpus_fname.split('/')[-1].split(')')[0]
        print('\ncorpus name = %s, num nouns = %d'%(corpus_name, len(nouns_dictionary)))
        
    with open(noun_dictionary_fname, 'w', encoding='utf-8') as f:
        for noun in nouns_dictionary:
            f.write('%s\n'%noun)
            
    print('done')

num corpus = 10
used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
2398 r features was loaded
scanning completed
(L,R) has (35278, 17088) tokens
building lr-graph completed
corpus name = 2016-10-22_article_all_normed.txt, num nouns = 10468
used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
2398 r features was loaded
scanning completed
(L,R) has (48684, 23844) tokens
building lr-graph completed
corpus name = 2016-10-23_article_all_normed.txt, num nouns = 16822
used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
2398 r features was loaded
scanning completed
(L,R) has (88431, 43874) tokens
building lr-graph completed
corpus name = 2016-10-25_article_all_normed.txt, num nouns = 28596
used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
2398 r features was loaded
scanning completed
(L,R) has (91374, 44957) tokens
building lr-graph completed
corpus name = 2016-10-24_article_all_no

2016-10-20일 뉴스에 대하여 CountVectorizer를 이용하여 term frequency matrix를 만든 뒤, 그 날의 주요 뉴스들이 어떤 것들이 있었는지 군집화를 수행함. 
- KMeans는 sparse matrix로 군집화를 수행할 수 있음

cutom_tokenizer를 이용함. noun_dictionary는 텍스트 파일로 저장되어 있으므로, 파일을 open할 때는 encoding='utf-8'로 설정해야함

In [3]:
with open(noun_dictionary_fname, encoding='utf-8') as f:
    nouns_dictionary = [noun.strip() for noun in f]
    print(len(nouns_dictionary))
    
def custom_tokenize(doc):
    def parse_noun(token):
        for e in reversed(range(1, len(token) + 1)):
            subword = token[:e]
            if subword in nouns_dictionary:
                return subword
        return ''
    
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns

custom_tokenize('국정농단의 사태에 대하여 뉴스 분석을 시작해봄')

58450


['국정농단', '사태', '뉴스', '분석', '시작']

2016-10-20 뉴스에 대하여 corpus를 만듦. corpus의 lenth로 확인

In [4]:
if TOKENIZE_2016_1020:
    import sys
    # from corpus import Corpus
    from soynlp.utils import DoublespaceLineCorpus as Corpus
    corpus_2016_1020 = Corpus(normed_corpus_fname, iter_sent=False)
    print('corpus length of 2016-10-20 = %d'%len(corpus_2016_1020))
    
    tokenized_corpus_2016_1020 = []
    for num_doc, doc in enumerate(corpus_2016_1020):
        if num_doc % 100 == 0:
            sys.stdout.write('\rtokenizing %d ...'%num_doc)
        doc = ' '.join([noun for sent in doc.split('  ') for noun in custom_tokenize(sent)]).strip()
        tokenized_corpus_2016_1020.append(doc)
    print('\rtokenization was done')
    
    with open(tokenized_corpus_fname, 'w', encoding='utf-8') as f:
        for doc in tokenized_corpus_2016_1020:
            f.write('%s\n'%doc)
            
else:
    with open(tokenized_corpus_fname, encoding='utf-8') as f:
        tokenized_corpus_2016_1020 = [doc.strip() for doc in f]

corpus length of 2016-10-20 = 30091
tokenization was done


CountVectorizer를 이용하여 term frequency matrix를 만듦. 만들어 둔 term frequency matrix는 corpus_10days/models/ 아래에 저장함

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

if CREATE_TERM_FREQUENCY_MATRIX:
    vectorizer = CountVectorizer(min_df=0.005, max_df=0.8)
    x_2016_1020 = vectorizer.fit_transform(tokenized_corpus_2016_1020)
    print(x_2016_1020.shape)
    
    from scipy.io import mmwrite
    mmwrite(x_2016_1020_fname, x_2016_1020)
    
    import pickle
    with open(vectorizer_2016_1020_fname, 'wb') as f:
        pickle.dump(vectorizer, f)
        
else:
    from scipy.io import mmread
    x_2016_1020 = mmread(x_2016_1020_fname).tocsr()
    
    import pickle
    with open(vectorizer_2016_1020_fname, 'rb') as f:
        vectorizer = pickle.load(f)

(30091, 2611)


x_2016\_1020의 각 column에 해당하는 단어를 decoding하기 위하여 vectorizer.vocabulary_로부터 int2vocab을 만듦

In [6]:
int2vocab = sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
int2vocab = [word for word, idx in int2vocab]
print(int2vocab[:5])

['00', '000', '01', '02', '03']


만들어진 x_2016_1020을 이용하여 Spherical kmeans를 수행. 이를 위해 L2로 normalization을 수행한 뒤, 클러스터의 갯수를 1000개로 지정하여 k-means clustering을 수행함

군집화나 토픽 모델링을 수행할 때는 예상하는 것보다 군집/토픽의 갯수를 크게 잡으면 됨. 중복되는 군집이 등장하면 묶으면 됨. 하지만, 데이터에 노이즈가 있으므로 다른 군집들이 하나의 군집으로 묶인다면 나중에 해석하기가 어려워짐

특히, 여러 군집/토픽에서 두루두루 사용될 수 있는 단어들이 이러한 노이즈 역할을 함. 이런 단어들을 미리 걸러낼 수 있다면 훨씬 더 정교한 모델링이 될 것임(군집화나 토픽모델링에서는 불필요하다 생각되는 단어들을 과감하게 쳐낼수록 결과가 깔끔하게 나타나는 경향이 있음). 그렇지 않드면, 군집화/토픽모델링을 할 때 군집/토픽의 갯수를 크게 잡아주면 더 좋음

군집화를 수행한 뒤, 20개의 군집들에 대해 각 군집에 할당된 뉴스의 갯수가 몇 개인지 출력

In [8]:
%%time

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from collections import defaultdict

x_2016_1020 = normalize(x_2016_1020, axis = 1, norm='l2')
kmeans = KMeans(n_clusters=1000, max_iter=20, n_init=1, verbose=-1)
clusters = kmeans.fit_predict(x_2016_1020)

clusters_to_rows = defaultdict(lambda: [])
for idx, label in enumerate(clusters):
    clusters_to_rows[label].append(idx)
    
for i in range(20):
    print('cluster # %d has %d docs'%(i, len(clusters_to_rows[i])))

Initialization complete
Iteration  0, inertia 16510.987
Iteration  1, inertia 13134.890
Iteration  2, inertia 12705.566
Iteration  3, inertia 12504.660
Iteration  4, inertia 12393.908
Iteration  5, inertia 12329.651
Iteration  6, inertia 12288.252
Iteration  7, inertia 12258.259
Iteration  8, inertia 12241.061
Iteration  9, inertia 12231.583
Iteration 10, inertia 12225.207
Iteration 11, inertia 12220.059
Iteration 12, inertia 12215.621
Iteration 13, inertia 12212.095
Iteration 14, inertia 12209.341
Iteration 15, inertia 12207.791
Iteration 16, inertia 12206.889
Iteration 17, inertia 12206.405
Iteration 18, inertia 12205.830
Iteration 19, inertia 12205.528
cluster # 0 has 34 docs
cluster # 1 has 19 docs
cluster # 2 has 22 docs
cluster # 3 has 854 docs
cluster # 4 has 59 docs
cluster # 5 has 106 docs
cluster # 6 has 52 docs
cluster # 7 has 26 docs
cluster # 8 has 46 docs
cluster # 9 has 9 docs
cluster # 10 has 1481 docs
cluster # 11 has 156 docs
cluster # 12 has 34 docs
cluster # 13 has 

군집 00번은 00개의 뉴스가 묶여 있음(군집화를 할 때마다 달라지므로, 다시 실행시킬 경우 다른 숫자로 나타날 것임). 이 군집에 대해 키워드를 추출하기 위해서는, 00번 군집에 해당하는 뉴스의 label을 1로, 다른 뉴스들을 -1로 둔 뒤, 해당 군집 00번을 구분할 수 있는 주요 단어들을 L1 regularized logistc regression을 이용하여 뽑아낼 수 있음

y를 만든 뒤, 혹시 하는 마음에 x_2016_1020.shape과 같은지 확인해보자

In [9]:
cluster_aspect = 15
y = [1 if i in clusters_to_rows[cluster_aspect] else -1 for i in range(x_2016_1020.shape[0])]
len(y), x_2016_1020.shape

(30091, (30091, 2611))

LogisticRegression의 penalty를 'l1'으로 주고, Regularization cost를 1로 주었음. 키워드의 갯수가 너무 많으면 C를 더 적게, 키워드의 갯수가 너무 적다면 C를 좀 더 크게 조절할 수 있음.

In [11]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=1)
logistic_l1.fit(x_2016_1020, y)

keywords = sorted(enumerate(logistic_l1.coef_[0]), key=lambda x:x[1], reverse=True)[:30]
for word, score in keywords:
    if score == 0: break
    print('%s (%.3f)' % (int2vocab[word], score))

마이데일리 (20.572)
사랑 (14.341)
영화 (5.431)


관심있는 군집들에 대해 키워드를 뽑아내는 함수를 생성
interested_clusters로 입력되는 클러스터들에 대하여 위처럼 각각의 클러스터에 대한
Regularized Logistic Regression을 이용하여 키워드를 추출

Regularization cost와 키워드의 갯수를 선택하기 위하여 print_keywords의 함수의 arguments에 이를 넣어둔다. L1 Logistic Regression에서 coefficient가 0이라는 것은 classification에서 유의미한 변수가 아니라는 뜻임. 그렇기 때문에 아래의 구문을 넣어서 coeffieicnt가 0보다 큰 단어들만 선택

    keywords = [ int2vocab[word] for word, score in keywords if score > 0 ]

단어 선택과 같은 튜닝 과정없이 명사 추출 + k-means clustering + 키워드 추출만으로도 어느 정도 그날의 뉴스 토픽들을 살펴볼 수 있음

좀 더 정확한 단어를 선택하여 군집화가 잘 되게 만들고, 키워드를 추출하기에 좋은 parameter를 찾아야 함

하지만 이러한 접근 방법에는 한 가지 단점이 존재함. 만약 18번 군집에 대한 키워드로 '디자이너'가 선택되었으나, 이 단어는 다른 군집에서의 키워드가 될 수 있음. 다른 여러개의 군집에서도 키워드로 사용될 수 있는 단어라면, 18번 군집과 비슷한 군집이 있기 때문임. 이는 topic modeling, doc2vec, 단어 선택을 통한 군집화의 고도화 등으로 해결해야 함

In [13]:
def print_keywords(interested_clusters, x, int2vocab, clusters_to_rows, C=1, topn=30):
    
    for cluster_id in interested_clusters:
        interested_docs = clusters_to_rows[cluster_id]
        print('\ncluster # %d keywords (from %d news)' % (cluster_id, len(interested_docs)))
        y = [1 if i in interested_docs else -1 for i in range(x_2016_1020.shape[0])]
        
        logistic_l1 = LogisticRegression(penalty='l1', C=C)
        logistic_l1.fit(x, y)

        keywords = sorted(enumerate(logistic_l1.coef_[0]), key=lambda x:x[1], reverse=True)[:topn]
        keywords = [int2vocab[word] for word, score in keywords if score > 0]
        print(' > %s' % (keywords))
        
interested_clusters = [i for i, labels in clusters_to_rows.items() if 50 < len(labels) < 500]
print_keywords(interested_clusters, x_2016_1020, int2vocab, clusters_to_rows, C=1.0)


cluster # 593 keywords (from 90 news)
 > ['투자', '글로벌', '상품', '미국', '설명', '국내', '유럽', '이상', '주식', '부동산']

cluster # 440 keywords (from 80 news)
 > ['상승', '증시', '시장', '예상', '금리', '지수', '실적', '대비', '거래', '강세', '발표', '미국', '이후', '전일', '전망']

cluster # 522 keywords (from 64 news)
 > ['대통령', '박근혜', '면서', '경북', '기록', '의장', '기존', '가방', '최순실', '지지율', '포인트', '서울신문']

cluster # 135 keywords (from 63 news)
 > ['연합뉴스', '대전', '의료', '제공', '대구', '구조', '20일', '부산']

cluster # 578 keywords (from 68 news)
 > ['북한', '우리', '송민순']

cluster # 707 keywords (from 170 news)
 > ['미국', '소장', '세계', '것으', '박사', '뉴욕', '보도', '자동차', '차량', '분석', '가능성', '소송', '지난', '현대차', '세금', '유럽', '이후', '국내', '영국', '주택', '공급', '금지', '시장', '최대', '문제', '생산', '보고']

cluster # 263 keywords (from 186 news)
 > ['경찰', '부검', '경찰관', '공격', '사건', '무단복제', '신고', '협의', '광주', '단체', '20대', '주장', '수사', '법원', '남성', '전국', '카드', '상황', '주변', '이런', '학교', '시스템', '차량', '관련', '총격', '사망', '시민들', '위반', '성능', '시민']

cluster # 432 keywords (from 90 news)
 > ['대

 > ['청와대', '거부', '이유', '야당', '사유서', '불출석', '수석', '국회', '대한', '운영위', '국정감사', '국감']

cluster # 538 keywords (from 88 news)
 > ['기자간담회', '대한', '지난', '방문', '뉴시스', '트레', '오전', '제보', '규모', '영상', '사장', '사무실', '검찰', '의혹', '서울', '스포츠재단', '대표', '촉구']

cluster # 239 keywords (from 135 news)
 > ['뉴시스', '제보', '전주', '영상', '협상', '화성', '수원', '파업', '기록']

cluster # 4 keywords (from 59 news)
 > ['정부서울청사', '종로구', '뉴시스', '사진', '기획재정부', '유일', '하반기']

cluster # 741 keywords (from 69 news)
 > ['고객', '혜택', '추가', '해외', '할인', '디지털', '가능', '마케팅', '투자자', '제출', '제공']

cluster # 312 keywords (from 103 news)
 > ['소재', '적용', '스타일', '착용', '브랜드', '사용', '디자인', '컬러', '날씨', '연출', '가능', '가을', '선택', '제품', '블랙', '라인', '효과']

cluster # 42 keywords (from 136 news)
 > ['김선', '뉴시스', '중구', '컬렉션', '서울패션위크', '디자이너', '사진', '동대문디자인플라자']

cluster # 36 keywords (from 62 news)
 > ['문재인', '뉴시스', '더불어민주당', '사진', '제3', '서울', '영상', '사람']

cluster # 968 keywords (from 53 news)
 > ['6시', '경찰', '출동', '총기', '총격', '피의자', '사제', '강북경찰서', '검거', '현장